In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1337)

In [2]:
lstm = nn.LSTM(input_size=3, hidden_size=3, num_layers=1) 
# stack 2 lstms if needed, but specify that many number of initial values of the hidden state
# check other arguments such as BIDIRECTIONAL and dropout 

inputs = [torch.randn(1, 3) for _ in range(5)] # create a sequence with length=5 

In [3]:
inputs

[tensor([[1.0517, 0.2501, 1.5759]]),
 tensor([[0.6495, 0.9631, 1.0905]]),
 tensor([[-0.4931, -0.9682, -0.1791]]),
 tensor([[ 0.2050, -0.7801, -1.0036]]),
 tensor([[ 0.0850,  1.0277, -0.3999]])]

In [4]:
# randomly initialize the states
hidden = (torch.randn(1,1,3), 
          torch.randn(1,1,3)) # c and h
# [batchsize x max_length x dim_size] ?

for i in inputs: # feed items in sequence, one by one. And compute hidden states
    out, hidden = lstm(i.view(1,1,-1), hidden)

In [5]:
hidden # c and h

(tensor([[[-0.2300,  0.1136,  0.1939]]], grad_fn=<ViewBackward>),
 tensor([[[-0.5540,  0.2028,  0.3885]]], grad_fn=<ViewBackward>))

In [6]:
out # y

tensor([[[-0.2300,  0.1136,  0.1939]]], grad_fn=<CatBackward>)

In [7]:
# alternatively, we can do the entire sequence all at once.
# the first output 'out' is the h at each timestep
# the second output 'hidden' is the h at the final timestep only

inputs = torch.cat(inputs).view(len(inputs), 1, -1)

# initialize hidden states
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))


In [8]:
inputs.shape # a single sequence of 5 words. 
# Note that in pytorch the second dim is the batch-size

torch.Size([5, 1, 3])

In [9]:
out, hidden = lstm(inputs, hidden)
print(out.shape) # h at each timestep
hidden  # the two items in the tuple corresponds to (h_N, c_N) 

torch.Size([5, 1, 3])


(tensor([[[-0.2393,  0.1125,  0.1946]]], grad_fn=<ViewBackward>),
 tensor([[[-0.5841,  0.1993,  0.3880]]], grad_fn=<ViewBackward>))

### Data Preparation

In [10]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


### LSTM Tagger Model

In [11]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(num_embeddings=vocab_size, 
                                            embedding_dim=embedding_dim
                                           )
        
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim)
        
        self.hidden2tag = nn.Linear(in_features=hidden_dim, 
                                    out_features=tagset_size)
        
        self.hidden = self.init_hidden() # reset hidden state after every input sequence
        
    def init_hidden(self):
        
        # random unif between these 2 values
        r1 = -0.05
        r2 = 0.05
        h_0 = (r1 - r2) * torch.rand(1, 1, self.hidden_dim) + r2
        c_0 = (r1 - r2) * torch.rand(1, 1, self.hidden_dim) + r2
        return (h_0, c_0)
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), 
                                          self.hidden)
        
        # collapse the middle 'batch-size' dimension
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        
        return tag_scores
    
    

### Model Training

In [12]:
model = LSTMTagger(embedding_dim=EMBEDDING_DIM,
                   hidden_dim=HIDDEN_DIM, 
                   vocab_size=len(word_to_ix),
                   tagset_size=len(tag_to_ix)
                  )

loss_function = nn.NLLLoss() 

optimizer = optim.SGD(model.parameters(), lr=0.1)

In [13]:
# before we start training
# have a look at the score based on the randomly initialized values
# in the output, [i, j] corresponds to the score for tag j while predicting for word i

with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-0.9864, -1.1653, -1.1543],
        [-1.0900, -1.1985, -1.0157],
        [-1.0345, -1.2150, -1.0560],
        [-0.9867, -1.2541, -1.0733],
        [-1.0916, -1.2166, -0.9994]])


In [14]:
for epoch in range(300):
    for sentence, tags in training_data:
        
        # clear gradients before each iteration
        model.zero_grad()
        
        # initialize hidden state
        model.hidden = model.init_hidden()
        
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        out_scores = model(sentence_in)
        
        loss = loss_function(out_scores, targets)
        loss.backward()
        optimizer.step()

### Inference Time

In [15]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix) 
    tag_scores = model(inputs)
    tag_scores = tag_scores.topk(k=1, dim=1)
    tags = tag_scores[1]

print(tags)

tensor([[0],
        [1],
        [2],
        [0],
        [1]])


In [16]:
tag_to_ix

{'DET': 0, 'NN': 1, 'V': 2}